In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()
# 메모리 터지는 문제를 사전에 방지하고자 최대 메모리 사용량을 작성 


22/10/06 19:13:07 WARN Utils: Your hostname, Moon-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
22/10/06 19:13:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 19:13:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 19:13:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/06 19:13:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
ratings_file = "/Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/ml-25m/ratings.csv"
ratings_df = spark.read.csv(f"file:///{ratings_file}", inferSchema=True, header=True)
# csv파일을 df로 변환해줌 

In [4]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
# userId, movieId, rating만 필요함
ratings_df = ratings_df.select(["userId", "movieId", "rating"])

In [6]:
print(ratings_df.show())
print(ratings_df.printSchema())

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows

None
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)

None


In [7]:
ratings_df.select("rating").describe().show()
# rating에 대한 컬럼의 통계값 

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [8]:
# train과 test 데이터 셋으로 분리 
train_df, test_df = ratings_df.randomSplit([0.8,0.2])

In [9]:
#ASL 알고리즘을 사용하기 위해 필요한 라이브러리 import
from pyspark.ml.recommendation import ALS

In [10]:
als = ALS(
    maxIter=5, 
    regParam=0.01,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy= "drop"
    # 학습하지 못한 데이터를 어떻게 처리해야하는지 여부 (Nan, drop)
         )

In [11]:
model = als.fit(train_df)
# fit으로 모델을 반환받음
# memory 오류가 발생하는데 이것을 해결하기 위해서 sparkSession에서 따로 설정을 해야함 

22/10/06 19:13:50 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/06 19:13:50 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/10/06 19:13:51 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
pred = model.transform(test_df)
pred.show()
# 예측 결과 
# rating이 진짜 값이고 prediction이 예측 값 

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    26|     50|   5.0| 3.9619906|
|    26|    110|   4.0| 3.6664298|
|    26|    589|   5.0| 3.1483727|
|    26|   1037|   2.5| 3.0810199|
|    26|   1991|   3.0|  2.876357|
|    26|  79132|   5.0| 3.9007967|
|    27|    527|   4.0| 4.1846704|
|    27|   1196|   2.0| 2.8358152|
|    27|   1247|   3.0| 3.7043045|
|    27|   1387|   1.0| 3.6237538|
|    27|   1721|   5.0|  4.125617|
|    27|   2570|   3.0| 3.6387498|
|    27|   2779|   3.0| 4.1945496|
|    27|   3263|   4.0| 3.3141344|
|    27|   3444|   2.0|  3.077427|
|    27|   5060|   5.0| 3.7880025|
|    28|    253|   5.0| 3.8962984|
|    28|    750|   5.0|  5.034409|
|    28|   1210|   5.0| 4.8852086|
|    28|   1270|   4.0| 5.1380053|
+------+-------+------+----------+
only showing top 20 rows



In [13]:
pred.select("rating", "prediction").describe().show()
# 실제값과 예측값의 통계값을 볼 수 있음 

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4994940|           4994940|
|   mean|3.5337071115969363| 3.420125133641283|
| stddev|1.0606407798086621|0.7257908683373027|
|    min|               0.5|        -10.020923|
|    max|               5.0|         12.915684|
+-------+------------------+------------------+



### Evaluation

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol= "prediction")
# 평가 내용을 정의 

In [15]:
rmse = evaluator.evaluate(pred)
# 실제 예측값을 가지고 평가 진행
print(rmse)

0.8176114035625365


In [16]:
model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{154634, 12.8814...|
|    27|[{175455, 17.7619...|
|    28|[{199340, 18.9276...|
|    31|[{175455, 10.0264...|
|    34|[{185367, 13.6254...|
|    44|[{112577, 14.9260...|
|    53|[{159131, 20.6558...|
|    65|[{112577, 17.9947...|
|    76|[{176179, 13.1805...|
|    78|[{174403, 15.9118...|
|    81|[{184501, 13.6648...|
|    85|[{175455, 18.0747...|
|   101|[{176179, 12.0474...|
|   103|[{95695, 16.83391...|
|   108|[{154634, 11.5487...|
|   115|[{116847, 16.5391...|
|   126|[{134037, 13.8541...|
|   133|[{176179, 11.8309...|
|   137|[{175455, 22.0059...|
|   148|[{176179, 13.5713...|
+------+--------------------+
only showing top 20 rows



In [17]:
 model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     12|[{123802, 6.77342...|
|     26|[{110647, 5.77255...|
|     27|[{63627, 6.233325...|
|     28|[{135583, 6.73147...|
|     31|[{52973, 5.605549...|
|     34|[{84066, 7.278187...|
|     44|[{109870, 7.08627...|
|     53|[{34628, 12.83511...|
|     65|[{67571, 7.183996...|
|     76|[{104240, 6.10939...|
|     78|[{61732, 7.156068...|
|     81|[{89022, 6.321851...|
|     85|[{135583, 7.03802...|
|    101|[{66931, 5.923635...|
|    103|[{9073, 6.3512564...|
|    108|[{149971, 8.74020...|
|    115|[{1225, 10.269253...|
|    126|[{34628, 6.757936...|
|    133|[{94231, 8.373791...|
|    137|[{112808, 8.03427...|
+-------+--------------------+
only showing top 20 rows



In [18]:
# 특정 유저만을 위한 추천을 할 수 있는 시스템 
from pyspark.sql.types import IntegerType

user_list = [65,78,81]
# 특정 유저 
# 여기서는 id가 65, 78, 81번 유저 

users_df = spark.createDataFrame(user_list, IntegerType()).toDF("userId")
# df로 변환하고 컬럼 이름 지정 
users_df.show()


+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [21]:
user_recoms = model.recommendForUserSubset(users_df, 5)
# df로 변환한 유저 |데이터를 통해 5개의 movie를 추천 
# 반환은 유저아이디와 movieId 

In [22]:
movies_list = user_recoms.collect()[0].recommendations
# 첫번째 유저의 추천 영화 목록 

In [23]:
movies_list
# spark df로 바꿔줘야함

[Row(movieId=112577, rating=17.99478530883789),
 Row(movieId=115864, rating=16.356884002685547),
 Row(movieId=149130, rating=15.305953979492188),
 Row(movieId=104774, rating=14.693916320800781),
 Row(movieId=181469, rating=14.422035217285156)]

In [24]:
recoms_df = spark.createDataFrame(movies_list)
recoms_df.show()
# 아이디를 이름값으로 변경해야함 

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 112577| 17.99478530883789|
| 115864|16.356884002685547|
| 149130|15.305953979492188|
| 104774|14.693916320800781|
| 181469|14.422035217285156|
+-------+------------------+



In [38]:
movies_file = "/Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/ml-25m/movies.csv"
movies_df = spark.read.csv(f"file:///{movies_file}", inferSchema=True, header=True)
# csv파일을 df로 변환해줌 

In [39]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [40]:
# sql에서 사용하기 위해 tempview 등록 
recoms_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

In [41]:
query = """
SELECT * 
FROM 
    movies JOIN recommendations 
    ON movies.movieId = recommendations.movieId
ORDER BY 
    rating desc
"""

recommended_movies = spark.sql(query)
recommended_movies.show()
# 특정 유저의 추천 영화 리스트 5개를 볼 수 있음 

+-------+--------------------+--------------------+-------+------------------+
|movieId|               title|              genres|movieId|            rating|
+-------+--------------------+--------------------+-------+------------------+
| 112577|Willie & Phil (1980)|Comedy|Drama|Romance| 112577| 17.99478530883789|
| 115864|  Hue and Cry (1947)| Action|Comedy|Crime| 115864|16.356884002685547|
| 149130|The Zohar Secret ...| Comedy|Drama|Sci-Fi| 149130|15.305953979492188|
| 104774|Three Worlds (Tro...|               Drama| 104774|14.693916320800781|
| 181469|Altered Minds (2014)|      Drama|Thriller| 181469|14.422035217285156|
+-------+--------------------+--------------------+-------+------------------+



In [42]:
# 추천을 serving하는 function 만들기 
def get_recommendations(user_id, num_recs):
    
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF("userId")
    # 유저의 정보를 받아서 df로 변경 
    
    user_recs_df = model.recommendForUserSubset(users_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations 
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
     
    return recommended_movies

In [44]:
recs = get_recommendations(456, 5)

In [46]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,112577,Willie & Phil (1980),Comedy|Drama|Romance,112577,17.994785
1,115864,Hue and Cry (1947),Action|Comedy|Crime,115864,16.356884
2,149130,The Zohar Secret (2015),Comedy|Drama|Sci-Fi,149130,15.305954
3,104774,Three Worlds (Trois mondes) (2012),Drama,104774,14.693916
4,181469,Altered Minds (2014),Drama|Thriller,181469,14.422035
